In [ ]:
import pandas as pd
import pickle
import ast
import numpy as np

from tqdm import tqdm
import os
import random
import re

os.getcwd()
tqdm.pandas()

from langchain_core.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp, Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler

sequence = pd.read_csv("/data/log-data-2024/SASRec/BSARec/src/data/input_search_sample_matching_final.txt") # past squence
match = pd.read_csv("/data/log-data-2024/1.meta_rot_preprocessing_ksc/data/Meta_V3.csv")
match = dict(zip(match["contents_id"], match["title_name"]))

samp = list(pd.read_csv("/data/log-data-2024/8man_sample_new_new.csv")["treatment2"]) # 1st sampling
sequence["use"] = [1 if i in samp else 0 for i in sequence["device_id"]]
sequence = sequence[sequence["use"] == 1].reset_index(drop = True)

with open(file='/data/log-data-2024/2.sequence_generate_ksc/data/content_meta_dict_final.pickle', mode='rb') as f:
    met = pickle.load(f) # meta

meta = {}
for i in met:
    meta[i.split(". ")[1]] = met[i]

samp = pd.read_csv("/data/log-data-2024/8man_sample_20241022_treatment2.csv") # 2nd sampling
samp["treatment2 prediction content_id"] = samp["treatment2 prediction content_id"].apply(lambda x : ast.literal_eval(x))
matched_samp = list(samp["treatment2"])
prediction = list(samp["treatment2 prediction content_id"])
sequence["use"] = [1 if i in matched_samp else 0 for i in sequence["device_id"]]
sequence = sequence[sequence["use"] == 1].reset_index(drop = True)

sequence["sequence"] = [i.split()[1:] for i in sequence["sequence"]]
sequence = dict(zip(sequence["device_id"], sequence["sequence"]))
samp["sequence"] = [sequence[i] for i in samp["treatment2"]]

samp = samp[["treatment2", "treatment2 prediction content_id", "sequence"]]
samp.columns = ["device_id", "prediction", "sequence"]

tt = []
for i in samp["prediction"]:
    ttt = []
    for w in i:
        ttt.append(match[w])
    tt.append(ttt)
samp["prediction"] = tt
    
sequence = samp.copy()

movie_meta = pd.read_csv("/data/meta/us_vod_metadata_movie.csv")
show_meta = pd.read_csv("/data/meta/us_vod_metadata_tvshow.csv")

movie_meta = movie_meta[["title_name", "age_code"]]
show_meta = show_meta[["title_name", "age_code"]]
add_meta = pd.concat([movie_meta, show_meta], axis = 0).reset_index(drop = True)

add_meta["title_name"] = add_meta["title_name"].apply(lambda x : x.upper())
add_meta["age_code"] = ["ALL" if i < 18 else "R19" for i in add_meta["age_code"]]
tn = dict(zip(add_meta["title_name"], add_meta["age_code"]))

for i in meta:
    try:
        meta[i] = meta[i][:-1] + ", age: {}".format(tn[i])
    except:
        pass
meta

In [ ]:
z = re.compile("genre_info: \[.+\], actor_info")
genres = []
for i in meta:
    try:
        temp = z.findall(meta[i])[0][13:-13].split(", ")
        for w in temp:
            genres.append(w)
    except:
        pass
genres = list(set(genres))
genre_list = []
for i in genres:
    genre_list.append(i)
    genre_list.append(i.title())
genre_list

In [ ]:
met = {}

for i in tqdm(meta):
    temp = i.split(". ")
    if len(temp) == 2:
        met[temp[0]] = "title: " + temp[1] + ", " + meta[i]
    else:
        met[temp[0]] = "title: " + " ".join(temp[1:]) + ", " + meta[i]

In [ ]:
os.environ["OLLAMA_LLM_LIBRARY"] = "cpu_avx2"

# model_name = 'llama3.1:70b'
model_name = 'llama3.1'

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = Ollama(
    model=model_name,
    temperature=0.75,
    verbose=False
)

In [ ]:
temp1 = []
temp2 = []
for a, i in enumerate(sequence["sequence"]):
    temp1.append(list(set(sequence["prediction"].loc[a])))
    temp2.append(list(set(sequence["prediction"].loc[a])))

In [ ]:
t1 = []
for i in tqdm(temp1):
    t2 = {}
    for w in i:
        try:
            t2[w] = met[str(w)]
        except:
            pass
    t1.append(t2)
sequence["meta"] = t1

t1 = []
for i in tqdm(temp2):
    t2 = "ALL"
    for w in i:
        try:
            if "R19" in met[str(w)]:
                t2 = "R19"
        except:
            pass
    t1.append(t2)
sequence["meta2"] = t1

In [ ]:
from collections import Counter
import heapq

g = []
for i in sequence["meta"]:
    gen = []
    for w in i:
        try:
            for p in z.findall(i[w])[0][13:-13].split(", "):
                gen.append(p)
        except:
            pass
    tops = heapq.nlargest(3, dict(Counter(gen)), key=dict(Counter(gen)).get)
    top = []
    for w in tops:
        top.append(w)
        top.append(w.title())
    g.append(top)
sequence["major_genre"] = g

In [ ]:
# few-shots
exp = pd.read_csv("classification.csv")
good_examples = list(exp[exp["Rating"] == 1]["message"])[:10]
bad_examples = list(exp[exp["Rating"] == 0]["message"])

In [ ]:
def cleanse(string):
        
    r = re.compile('\".*?\"')
    a = re.compile(".+:")
    
    try:
        string = r.findall(string)[0]
    except:
        pass

    string = re.sub(a, " ", string)
    return string.strip()

def generate_message(table):
    result = []
    
    system1 = '''
    <|begin_of_text|><<|start_header_id|>system<|end_header_id|>
    You are a marketing manager responsible to generate recommendation messages for smart TV recommendation system.
    <|eot_id|>
    '''

    user1 = '''
    <|begin_of_text|><<|start_header_id|>user<|end_header_id|>
    I will provide you "a squence of contents" that a specific customer has previously viewed.
    Summarize the smart TV contents consumption patterns of the customer in one sentence.
    Concentrate on the common points of the contents if possible.
    Simply output the summary only.
    You should retrieve the "information of the contents" to identify which contents are assigned to each number.
    A squence of contents: {sequence}
    Information of the contents: {meta}.
    <|eot_id|>
    '''

    system2 = '''
    <|begin_of_text|><<|start_header_id|>system<|end_header_id|>
    You are a marketing manager responsible to generate recommendation messages for smart TV recommendation system.
    <|eot_id|>
    '''

    user2 = '''
    <|begin_of_text|><<|start_header_id|>user<|end_header_id|>
    Based on the following "characteristics of a customer", generate a recommendation message that encourage customers to click-through the recommended contents.
    The recommendation message should be a sentence consists of 15 - 60 characters.
    The recommended contents are already determined: you should not recommend specific contents or genres.
    Focus only on the characteristics of the customer, instead of focusing on the recommended contents.
    You should use only general nouns when referring to recommended contents, such as contents, programs, recommendations, and picks.
    Any modifier for the recommended contents is strictly prohibited. For example, recommending "triller contents" or "comedy programs" is not allowed.
    Characteristics of the customers: {character}
    Simply output one recommendation message only.
    Avoid generating similar messages with "Bad examples".

    Good examples: {good exp}
    Bad examples: {bad exp}
    <|eot_id|>
    '''

    for i in tqdm(table.index):

        template_text1 = system1 + user1

        prompt1 = PromptTemplate.from_template(template_text1)

        formatted_inputs1 = {
            "sequence": table["prediction"].loc[i],
            "meta": table["meta"].loc[i]
        }

        chain1 = prompt1 | llm | StrOutputParser()
        response1 = chain1.invoke(formatted_inputs1)

        # ------------------------------------------------------------------------

        template_text2 = system2 + user2

        prompt2 = PromptTemplate.from_template(template_text2)

        formatted_inputs2 = {
            "character": response1,
            "good exp" : good_examples,
            "bad exp" : bad_examples
        }

        chain2 = prompt2 | llm | StrOutputParser()
        response2 = chain2.invoke(formatted_inputs2)

        result.append(response2)
        
    df = pd.DataFrame({"device_id" : table["device_id"], "message" : result, "sequence" : table["sequence"],
                       "prediction" : table["prediction"], "meta" : table["meta"], "major_genre" : table["major_genre"]})
    
    df["message"] = df["message"].apply(lambda x : cleanse(x))
    return df
    
def approve_message(table):
    
    system = '''
    <|begin_of_text|><<|start_header_id|>system<|end_header_id|>
    You are a classifier.
    <|eot_id|>
    '''
    user = '''
    <|begin_of_text|><<|start_header_id|>user<|end_header_id|>
    I will provide you a recommendation message. Evaluate whether the message meets the conditions below.
    Output "Approved" if the message meets all conditions below, otherwise "Rejected".
    Output the basis of the decision with the decision per condition.
    Message: {msg}
    Condition 1. The message should not recommend specific contents or genres.
    Condition 2. Recommending contents to specific type of customers is allowed.
    Condition 3: The message should be grammatically correct and fluent.
    Condition 4: The message should be a sentence consists of 15 - 60 characters.
    Condition 5. Message recommending family-friendly contents is prohibited if the {meta} is "R19".
    Do not check other conditions than listed.
    Genres are listed in {grs}
    <|eot_id|>
    '''

    template_text = system + user

    prompt = PromptTemplate.from_template(template_text)

    filtering = []
    for i in table.index:
        classify = 1
        target = table["message"].loc[i]
        comp = table["major_genre"].loc[i]
        if ("Thrill" in target) & ("thriller" not in comp):
            classify = 0
        elif ("thrill" in target) & ("thriller" not in comp):
            classify = 0
        else:
            compare = list(set(genre_list) - set(comp))

            for w in compare:
                if w in target:
                    classify = 0
                    break
                else:
                    pass
        if classify == 1:
            filtering.append(1)
        else:
            filtering.append(0)
            
    result = []
    for a, i in tqdm(enumerate(table.index)):
        if filtering[a] == 0:
            result.append(0)
        else:
            formatted_inputs = {
                "msg" : target,
                "grs" : genres,
                "meta": table["meta"].loc[i]
            }

            chain = prompt | llm | StrOutputParser()
            response = chain.invoke(formatted_inputs)

            result.append(response)
            
    table["class"] = result
    table["class"] = [0 if i == 0 else 0 if "Rejected" in i else 0 if (("Rejected" not in i) & ("Approved" not in i)) else 1 for i in table["class"]]
    
    return table

In [ ]:
result = pd.DataFrame(columns = ["device_id", "message", "sequence", "prediction", "meta", "class"])
seq = sequence.copy()
df = seq.copy()

while len(result.index) != len(seq.index):
    df = generate_message(df)
    df = approve_message(df)
    
    approved = df[df["class"] == 1]
    df = df[df["class"] == 0]
    
    bad_examples = list(df["message"].iloc[:10])
    
    result = pd.concat([result, approved], axis = 0).reset_index(drop = True)
    
    if len(result.index) > len(sequence.index)/3:
        result.to_csv("result_20241118_llmemb_able.csv")
        break
    
result